In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [2]:
df1 = pd.read_csv("data/train.csv")
df2 = pd.read_csv("data/test.csv")
display(df1)
display(df2)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


In [3]:
df1.shape, df1.drop_duplicates().shape # 중복 존재

((2952, 15), (2632, 15))

In [4]:
df2.shape, df2.drop_duplicates().shape # 중복 존재

((1022, 14), (949, 14))

In [5]:
# 중복치 제거
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()
display(df1)
display(df2)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


In [6]:
# 아파트 값 제거 
df_apt = df1['임대건물구분'] != '아파트'
df1 = df1[df_apt]
display(df1)

df_apt1 = df2['임대건물구분'] != '아파트'
df2 = df2[df_apt1]
display(df2)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
88,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
91,C1925,601,상가,강원도,임대상가,72.16,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
101,C1874,619,상가,충청남도,임대상가,12.62,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
102,C1874,619,상가,충청남도,임대상가,17.40,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
104,C1874,619,상가,충청남도,임대상가,22.89,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,C2212,697,상가,강원도,임대상가,31.84,1,0.0,D,NaN,NaN,0.0,4.0,128.0,86.0
817,C2212,697,상가,강원도,임대상가,63.68,1,0.0,D,NaN,NaN,0.0,4.0,128.0,86.0
821,C2212,697,상가,강원도,임대상가,137.49,1,0.0,D,NaN,NaN,0.0,4.0,128.0,86.0
826,C2571,481,상가,경상남도,임대상가,31.84,1,5.0,D,NaN,NaN,0.0,1.0,65.0,62.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
80,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,NaN,NaN,2.0,5.0,428.0
82,C1006,1505,상가,대전광역시,임대상가,37.26,1,27.0,D,NaN,NaN,2.0,5.0,428.0
83,C1006,1505,상가,대전광역시,임대상가,37.41,1,27.0,D,NaN,NaN,2.0,5.0,428.0
86,C1006,1505,상가,대전광역시,임대상가,37.49,1,27.0,D,NaN,NaN,2.0,5.0,428.0
87,C1006,1505,상가,대전광역시,임대상가,37.95,1,27.0,D,NaN,NaN,2.0,5.0,428.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,C1729,627,상가,강원도,임대상가,32.46,1,0.0,D,NaN,NaN,0.0,5.0,131.0
329,C1729,627,상가,강원도,임대상가,34.86,1,0.0,D,NaN,NaN,0.0,5.0,131.0
330,C1729,627,상가,강원도,임대상가,35.76,1,0.0,D,NaN,NaN,0.0,5.0,131.0
331,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,NaN,0.0,5.0,131.0


In [7]:
강원 = pd.read_csv("울산_경남_강원/공공임대주택단지_강원_아파트 제외_utf8.csv")
경남 = pd.read_csv("울산_경남_강원/공공임대주택단지_경남_아파트 제외_utf8.csv")
울산 = pd.read_csv("울산_경남_강원/공공임대주택단지_울산_아파트 제외_utf8.csv")
display(강원)
display(경남)
display(울산)

강원 = 강원.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})
경남 = 경남.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})
울산 = 울산.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})

display(강원)
display(경남)
display(울산)

,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,광역시도 명,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,공급전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,0,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,47,47.30,0.00,다가구주택,0,0,0,NaN
1,1,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,49,49.24,0.00,다가구주택,0,0,0,NaN
2,2,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,49,49.59,0.00,다가구주택,0,0,0,NaN
3,3,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,73,73.62,0.00,다가구주택,0,0,0,NaN
4,4,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,77,77.95,0.00,다가구주택,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1740,2016,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,20,20.96,6.76,다가구주택,0,2471000,44350,NaN
1741,2017,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,20,20.96,9.22,다가구주택,0,2746000,51300,NaN
1742,2018,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,30,30.65,9.89,다가구주택,0,3661000,64620,NaN
1743,2019,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,30,30.65,13.47,다가구주택,0,4027000,74760,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,광역시도 명,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,공급전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,0,665,31370223,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 의안로12번길 31,13,매입임대,29,29.760,11.0300,다가구주택,0,2000000,154600,NaN
1,1,665,31370223,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 의안로12번길 31,13,매입임대,40,40.300,14.9300,다가구주택,0,2000000,190980,NaN
2,2,665,31370223,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 의안로12번길 31,13,매입임대,41,41.540,15.3900,다가구주택,0,2000000,201250,NaN
3,6,665,31325840,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 명서로120번길 9,4,매입임대,57,57.135,6.1100,다가구주택,0,0,0,NaN
4,7,665,31325840,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 명서로120번길 9,4,매입임대,57,57.248,6.1300,다가구주택,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,4102,466,30700760,LH경남,48,경상남도,330,양산시,"신축다세대(2011,양산삼호)",경상남도 양산시 삼호동부6길 14,16,매입임대,58,58.750,14.2375,다가구주택,0,60533000,0,NaN
3308,4166,7,31346042,LH경남,48,경상남도,870,함양군,경상남도 함양군,경상남도 함양군 서하면 송계앞길 7-2,14,매입임대,49,49.190,15.3000,다가구주택,0,6293000,95650,NaN
3309,4167,7,31346042,LH경남,48,경상남도,870,함양군,경상남도 함양군,경상남도 함양군 서하면 송계앞길 7-2,14,매입임대,73,73.490,15.5600,다가구주택,0,8751000,207330,NaN
3310,4168,7,31346042,LH경남,48,경상남도,870,함양군,경상남도 함양군,경상남도 함양군 서하면 송계앞길 7-2,14,매입임대,82,82.946,0.0000,다가구주택,0,0,0,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,광역시도 명,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,공급전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,0,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,25,25.080,5.024,다가구주택,0,2949000,73890,NaN
1,1,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,25,25.600,7.318,다가구주택,0,3146000,75510,NaN
2,2,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,27,27.405,7.834,다가구주택,0,3441000,80380,NaN
3,3,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,27,27.540,4.972,다가구주택,0,3146000,81390,NaN
4,4,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,27,27.665,7.908,다가구주택,0,3441000,81350,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,2536,310,30844206,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 모두박1길 8-6,8,매입임대,118,118.780,21.380,다가구주택,0,12902000,371180,NaN
2456,2537,310,30844205,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 대리3길 6-9,7,매입임대,42,42.420,6.590,다가구주택,0,4561000,149060,NaN
2457,2538,310,30844205,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 대리3길 6-9,7,매입임대,46,46.080,7.160,다가구주택,0,4934000,158170,NaN
2458,2539,310,30844205,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 대리3길 6-9,7,매입임대,47,47.400,7.360,다가구주택,0,5026000,165040,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,지역,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,0,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,47,47.30,0.00,다가구주택,0,0,0,NaN
1,1,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,49,49.24,0.00,다가구주택,0,0,0,NaN
2,2,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,49,49.59,0.00,다가구주택,0,0,0,NaN
3,3,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,73,73.62,0.00,다가구주택,0,0,0,NaN
4,4,738,31344091,LH강원,42,강원도,110,춘천시,강원도 춘천시,강원도 춘천시 동내면 학곡서3길 8-16,9,매입임대,77,77.95,0.00,다가구주택,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1740,2016,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,20,20.96,6.76,다가구주택,0,2471000,44350,NaN
1741,2017,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,20,20.96,9.22,다가구주택,0,2746000,51300,NaN
1742,2018,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,30,30.65,9.89,다가구주택,0,3661000,64620,NaN
1743,2019,95,31114286,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 구미3길 33-1,14,매입임대,30,30.65,13.47,다가구주택,0,4027000,74760,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,지역,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,0,665,31370223,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 의안로12번길 31,13,매입임대,29,29.760,11.0300,다가구주택,0,2000000,154600,NaN
1,1,665,31370223,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 의안로12번길 31,13,매입임대,40,40.300,14.9300,다가구주택,0,2000000,190980,NaN
2,2,665,31370223,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 의안로12번길 31,13,매입임대,41,41.540,15.3900,다가구주택,0,2000000,201250,NaN
3,6,665,31325840,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 명서로120번길 9,4,매입임대,57,57.135,6.1100,다가구주택,0,0,0,NaN
4,7,665,31325840,LH경남,48,경상남도,121,창원시 의창구,경상남도 창원시,경상남도 창원시 의창구 명서로120번길 9,4,매입임대,57,57.248,6.1300,다가구주택,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,4102,466,30700760,LH경남,48,경상남도,330,양산시,"신축다세대(2011,양산삼호)",경상남도 양산시 삼호동부6길 14,16,매입임대,58,58.750,14.2375,다가구주택,0,60533000,0,NaN
3308,4166,7,31346042,LH경남,48,경상남도,870,함양군,경상남도 함양군,경상남도 함양군 서하면 송계앞길 7-2,14,매입임대,49,49.190,15.3000,다가구주택,0,6293000,95650,NaN
3309,4167,7,31346042,LH경남,48,경상남도,870,함양군,경상남도 함양군,경상남도 함양군 서하면 송계앞길 7-2,14,매입임대,73,73.490,15.5600,다가구주택,0,8751000,207330,NaN
3310,4168,7,31346042,LH경남,48,경상남도,870,함양군,경상남도 함양군,경상남도 함양군 서하면 송계앞길 7-2,14,매입임대,82,82.946,0.0000,다가구주택,0,0,0,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,지역,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,0,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,25,25.080,5.024,다가구주택,0,2949000,73890,NaN
1,1,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,25,25.600,7.318,다가구주택,0,3146000,75510,NaN
2,2,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,27,27.405,7.834,다가구주택,0,3441000,80380,NaN
3,3,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,27,27.540,4.972,다가구주택,0,3146000,81390,NaN
4,4,648,31320260,LH부산울산,31,울산광역시,110,중구,울산광역시 중구,울산광역시 중구 해오름18길 28,33,매입임대,27,27.665,7.908,다가구주택,0,3441000,81350,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,2536,310,30844206,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 모두박1길 8-6,8,매입임대,118,118.780,21.380,다가구주택,0,12902000,371180,NaN
2456,2537,310,30844205,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 대리3길 6-9,7,매입임대,42,42.420,6.590,다가구주택,0,4561000,149060,NaN
2457,2538,310,30844205,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 대리3길 6-9,7,매입임대,46,46.080,7.160,다가구주택,0,4934000,158170,NaN
2458,2539,310,30844205,LH부산울산,31,울산광역시,710,울주군,울산광역시 울주군,울산광역시 울주군 범서읍 대리3길 6-9,7,매입임대,47,47.400,7.360,다가구주택,0,5026000,165040,NaN


In [8]:
# 결측값 : 경남 강원
경남지역 = df1['지역'] == '경상남도'
df1경남 = df1[경남지역]

강원지역 = df1['지역'] == '강원도'
df1강원 = df1[강원지역]

display(df1경남)
display(df1강원)

merge_df1경남 = pd.merge(df1경남, 경남, on='전용면적')
merge_df1경남.to_csv('지역별합친데이터/트레인_경남_merge_cp949.csv', encoding='cp949')


merge_df1강원 = pd.merge(df1강원, 강원, on='전용면적')
merge_df1강원.to_csv('지역별합친데이터/트레인_강원_merge_cp949.csv', encoding='cp949')

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
133,C2416,560,상가,경상남도,임대상가,19.69,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
134,C2416,560,상가,경상남도,임대상가,27.12,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
135,C2416,560,상가,경상남도,임대상가,32.54,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
136,C2416,560,상가,경상남도,임대상가,36.43,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
137,C2416,560,상가,경상남도,임대상가,46.89,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
138,C2416,560,상가,경상남도,임대상가,49.99,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
139,C2416,560,상가,경상남도,임대상가,51.14,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
140,C2416,560,상가,경상남도,임대상가,317.17,1,0.0,D,NaN,NaN,0.0,5.0,154.0,63.0
319,C2190,1013,상가,경상남도,임대상가,16.00,1,7.0,D,NaN,NaN,0.0,2.0,178.0,129.0
320,C2190,1013,상가,경상남도,임대상가,14.17,1,7.0,D,NaN,NaN,0.0,2.0,178.0,129.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
88,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
91,C1925,601,상가,강원도,임대상가,72.16,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
306,C1722,809,상가,강원도,임대상가,32.10,1,1.0,D,NaN,NaN,0.0,6.0,264.0,649.0
311,C1722,809,상가,강원도,임대상가,72.16,1,1.0,D,NaN,NaN,0.0,6.0,264.0,649.0
649,C1206,755,상가,강원도,임대상가,31.84,1,1.0,D,NaN,NaN,0.0,4.0,88.0,90.0
657,C1206,755,상가,강원도,임대상가,63.68,1,1.0,D,NaN,NaN,0.0,4.0,88.0,90.0
658,C1206,755,상가,강원도,임대상가,137.49,1,1.0,D,NaN,NaN,0.0,4.0,88.0,90.0
811,C2212,697,상가,강원도,임대상가,31.84,1,0.0,D,NaN,NaN,0.0,4.0,128.0,86.0
817,C2212,697,상가,강원도,임대상가,63.68,1,0.0,D,NaN,NaN,0.0,4.0,128.0,86.0
821,C2212,697,상가,강원도,임대상가,137.49,1,0.0,D,NaN,NaN,0.0,4.0,128.0,86.0


In [9]:
# 결측값 : 강원 울산
강원지역 = df2['지역'] == '강원도'
df2강원 = df2[강원지역]

울산지역 = df2['지역'] == '울산광역시'
df2울산 = df2[울산지역]

merge_df2강원 = pd.merge(df2강원, 강원, on='전용면적')
merge_df2강원.to_csv('지역별합친데이터/테스트_강원_merge_cp949.csv', encoding='cp949')

merge_df2울산 = pd.merge(df2울산, 울산, on='전용면적')
merge_df2울산.to_csv('지역별합친데이터/테스트_울산_merge_cp949.csv', encoding='cp949')


display(merge_df2강원)
display(merge_df2울산)

,단지코드,총세대수,임대건물구분,지역_x,공급유형_x,전용면적,전용면적별세대수,공가수,자격유형,임대보증금_x,...,주소,세대수,공급유형_y,형 명,공급공용면적,주택유형,주차수,임대보증금_y,월임대료,준공일자
0,C2253,1161,상가,강원도,임대상가,55.50,1,0.0,D,NaN,...,강원도 원주시 판부면 오성마을길 33-4,5,매입임대,55,7.9500,다가구주택,0,5936000,79160,NaN
1,C1729,627,상가,강원도,임대상가,19.08,1,0.0,D,NaN,...,강원도 춘천시 효자문길 31,16,매입임대,19,15.2300,단독주택,0,5224000,121920,NaN
2,C1729,627,상가,강원도,임대상가,23.40,1,0.0,D,NaN,...,강원도 동해시 감추6길 23-6,11,매입임대,23,4.7462,다가구주택,0,2562000,44610,NaN
3,C1729,627,상가,강원도,임대상가,34.86,1,0.0,D,NaN,...,강원도 원주시 관설안길 9-3,8,매입임대,34,9.0700,다가구주택,0,4041000,56740,NaN
4,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,...,강원도 원주시 나비허리길 5,1,매입임대,50,18.2493,오피스텔,0,2000000,272630,NaN
5,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,...,강원도 원주시 나비허리길 5,1,매입임대,50,18.2493,오피스텔,0,2000000,272630,NaN
6,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,...,강원도 원주시 나비허리길 5,1,매입임대,50,18.2493,오피스텔,0,2000000,272630,NaN
7,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,...,강원도 원주시 나비허리길 5,1,매입임대,50,18.2493,오피스텔,0,2000000,272630,NaN
8,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,...,강원도 원주시 나비허리길 5,1,매입임대,50,18.2493,오피스텔,0,12088000,272630,NaN
9,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,...,강원도 원주시 나비허리길 5,1,매입임대,50,18.2493,오피스텔,0,2000000,272630,NaN


,단지코드,총세대수,임대건물구분,지역_x,공급유형_x,전용면적,전용면적별세대수,공가수,자격유형,임대보증금_x,...,주소,세대수,공급유형_y,형 명,공급공용면적,주택유형,주차수,임대보증금_y,월임대료,준공일자
0,C2033,1400,상가,울산광역시,임대상가,26.25,1,12.0,D,NaN,...,울산광역시 북구 호계1길 15-1,8,매입임대,26,5.770,다가구주택,0,2978000,75140,NaN
1,C2033,1400,상가,울산광역시,임대상가,28.50,1,12.0,D,NaN,...,울산광역시 중구 내황1길 10,7,매입임대,28,7.900,다가구주택,0,3264000,67000,NaN
2,C2033,1400,상가,울산광역시,임대상가,28.50,1,12.0,D,NaN,...,울산광역시 남구 신남로95번길 4-1,14,매입임대,28,4.112,다가구주택,0,2835000,86070,NaN
3,C2033,1400,상가,울산광역시,임대상가,28.50,1,12.0,D,NaN,...,울산광역시 북구 호계2길 66,9,매입임대,28,4.880,다가구주택,0,3072000,74020,NaN
4,C2033,1400,상가,울산광역시,임대상가,28.50,1,12.0,D,NaN,...,울산광역시 울주군 언양읍 서문3길 13,12,매입임대,28,3.970,다가구주택,0,3168000,88430,NaN
5,C2033,1400,상가,울산광역시,임대상가,28.50,1,12.0,D,NaN,...,울산광역시 울주군 온산읍 신경2길 45,8,매입임대,28,5.520,다가구주택,0,3191000,59030,NaN
6,C2033,1400,상가,울산광역시,임대상가,33.15,1,12.0,D,NaN,...,울산광역시 남구 돋질로445번길 8,7,매입임대,33,4.940,다가구주택,0,3659000,115310,NaN
7,C2033,1400,상가,울산광역시,임대상가,33.31,1,12.0,D,NaN,...,울산광역시 남구 중앙로220번길 20,7,매입임대,33,6.614,다가구주택,0,3428000,133540,NaN
8,C2033,1400,상가,울산광역시,임대상가,33.39,1,12.0,D,NaN,...,울산광역시 동구 번덕2길 41,9,매입임대,33,4.620,다가구주택,0,3667000,94720,NaN
9,C2033,1400,상가,울산광역시,임대상가,33.51,1,12.0,D,NaN,...,울산광역시 중구 해오름18길 28,33,매입임대,33,9.579,다가구주택,0,4228000,98160,NaN
